# Import Libraries and Startup Necessities

In [ ]:
import pandas as pd
from tqdm import tqdm, trange
import numpy as np
import os
import gc
import time

from matplotlib import pyplot as plt
from skimage.transform import resize
import seaborn as sns

from keras.applications.mobilenet import preprocess_input as preprocess_input_v1, decode_predictions as decode_predictions_v1
from keras.utils import load_img, img_to_array, to_categorical
from keras.applications import MobileNet
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam, AdamW, RMSprop, SGD
from keras.losses import CategoricalCrossentropy
# from keras.metrics import Accuracy
from tensorflow.keras.metrics import F1Score
from keras.callbacks import EarlyStopping, LearningRateScheduler
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

import wandb
from wandb.keras import WandbCallback

## Initialize Wandb

Use Kaggle Secret to store my Wandb API

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_jalal")

Login to Wandb using my own Wandb API Key stored on Kaggle Secret

In [ ]:
wandb.login(key=secret_value_0)

# Loading Dataset and Exploratory Data Analysis (EDA)

In [ ]:
ALL_NAMES = "/kaggle/input/lfw-dataset/lfw_allnames.csv"
IMAGES_SRC = "/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled"
PEOPLE = "/kaggle/input/lfw-dataset/people.csv"
PEOPLE_TEST = "/kaggle/input/lfw-dataset/peopleDevTest.csv"
PEOPLE_TRAIN = "/kaggle/input/lfw-dataset/peopleDevTrain.csv"
README = "/kaggle/input/lfw-dataset/lfw_readme.csv"
WORK_DIR = "/kaggle/working/"

In [ ]:
df_all = pd.read_csv(ALL_NAMES)
df_all = df_all.sort_values(by="images", ascending=False) # Descending
df_all.describe()

As we can see on the dataframe description above, the maximum data is 530 but the 3rd quartile is 2. This indicate a highly imbalance data. So, I would like to filter only use the data whose images count is > 70

In [ ]:
df_all = df_all[df_all["images"] > 70]

In [ ]:
unique_name_count = df_all['name'].nunique()

# Display the count of unique values in the "name" column
print("Count of unique names:", unique_name_count)
print("Count of all data:", df_all["images"].sum())

As we can see, this only gives us 7 classes/names after filtering, with total data 1288.

# Data Preprocessing

Loading the data set. This step follows the reference: https://keras.io/api/applications/#classify-imagenet-classes-with-resnet50.

In [ ]:
X = []
Y = []

tqdm_all_names = tqdm(list(df_all["name"]), colour="blue")

for name in tqdm_all_names:
    dir_path = os.path.join(IMAGES_SRC, name)
    list_images_name = os.listdir(dir_path)
    
    for image_name in list_images_name:
        images_path = os.path.join(dir_path, image_name)
        rgb_image = load_img(images_path, target_size=(224, 224))
        x = img_to_array(rgb_image)
        x = np.expand_dims(x, axis=0)
        x_v1 = preprocess_input_v1(x)
        X.append(x_v1)
        Y.append(name)

X = np.asarray(X)
Y = np.asarray(Y)

## Label Encoding

We use one-hot encoding to transform out label from text to number so the machine could understand it. One-hot encoder is used because the MobileNet requires for each label to be a list of its classes, much like sparse-encoding.

In [ ]:
encoder = OneHotEncoder(dtype=np.float32)

In [ ]:
# Fit and transform the data
encoded_Y = encoder.fit_transform(Y.reshape(-1, 1))
encoded_Y = encoded_Y.toarray()

In [ ]:
encoded_Y.shape

## Train-Dev-Test Split

Train : Dev : Test = 70 : 15 : 15

I tried to use stratify=Y_temp on `X_dev, X_test, Y_dev, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=30, stratify=Y_temp)` but it showed error "The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2."

So I use the regular `train_test_split` function from Scikit-learn.

In [ ]:
X1_train, X1_temp, Y1_train, Y1_temp = train_test_split(X, encoded_Y, test_size=0.3, random_state=30)
X1_dev, X1_test, Y1_dev, Y1_test = train_test_split(X1_temp, Y1_temp, test_size=0.5, random_state=30)

Next, I try to reshape the resulting dimension to be of (n_samples, 224, 224, 3).

In [ ]:
NEW_SHAPE = (-1, 224, 224, 3)
X1_train = X1_train.reshape(NEW_SHAPE)
X1_dev = X1_dev.reshape(NEW_SHAPE)
X1_test = X1_test.reshape(NEW_SHAPE)
print("X1_train shape:", X1_train.shape)
print("X1_dev shape:", X1_dev.shape)
print("X1_test shape:", X1_test.shape)

In [ ]:
print("Y1_train shape:", Y1_train.shape)
print("Y1_dev shape:", Y1_dev.shape)
print("Y1_test shape:", Y1_test.shape)

# AI Modelling

## Model Declaration

For the base model, I use MobileNet with "imagenet" as the initial weights. The base model is loaded without its classifier head.

In [ ]:
mobilenet_mdl = MobileNet(
    include_top=False,
    input_shape=(224, 224, 3),
    weights="imagenet",
)

Then, add a custom classifier head

In [ ]:
def head_model(base_model, n_classes):
    top_model = base_model.output
    top_model = GlobalAveragePooling2D(data_format="channels_last")(top_model)
    top_model = Dense(1024, activation="relu")(top_model)
    top_model = Dropout(0.69)(top_model)
    top_model = Dense(1000, activation="relu")(top_model)
    top_model = Dropout(0.55)(top_model)
    top_model = Dense(n_classes, activation="softmax")(top_model)
    return top_model

In [ ]:
# def head_model2(base_model, n_classes):
#     out_model = Sequential()
#     out_model.add(base_model)
#     out_model.add(GlobalAveragePooling2D(data_format="channels_last"))
#     out_model.add(Dense(2048, activation="relu"))
#     out_model.add(Dropout(0.35))
#     out_model.add(Dense(1024, activation="relu"))
#     out_model.add(Dropout(0.25))
#     out_model.add(Dense(n_classes, activation="softmax"))
#     return out_model

Stack the classifier head on the top of the base model

In [ ]:
model_mobilenet_head = head_model(mobilenet_mdl, encoded_Y.shape[1])
model_mobilenet = Model(inputs = mobilenet_mdl.input, outputs = model_mobilenet_head)

Freeze other than the top 6 layers (or, train the classifier heads), so that the trainable neurons is the one on the custom classifier head.

In [ ]:
for layer in model_mobilenet.layers[:-6]:
    layer.trainable = False

Look at the declared model architecture

In [ ]:
model_mobilenet.summary()

We can see the trainable params/neurons above is the same as the sum of the number of neurons from the last 6 layers. This indicates that we have freezed the base model.

## Training

### Helper Functions

Create helper function to plot the training process

In [ ]:
def plot_training_history(history):
    # Plot training and validation accuracy values
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['f1_score'], label='Training')
    plt.plot(history.history['val_f1_score'], label='Validation')
    plt.title('Training and Validation F1 Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()

    # Plot training and validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

Create helper function to warmup learning rate while training. This help the model to utilize high learning rate for a few epochs, then exponentially decrease the learning rate.

In [ ]:
def scheduler(running_epoch, learning_rate):
    try:
        if running_epoch <= 10:
            return learning_rate
        else:
            if learning_rate < float(1e-6): # Do NOT make the learning rate any lower than 1e-6
                return learning_rate
            else:
                return learning_rate * tf.math.exp(-0.1)
    except Exception as e:
        print(f"Error while returning the scheduler LR!", e)

Create helper function to show time elapsed

In [ ]:
def training_time_elapsed(elapsed_time):
    # Format the elapsed time
    hours, remainder = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(remainder, 60)

    if hours >= 1:
        print(f"The training goes by {int(hours)} hour, {int(minutes)} minutes, and {int(seconds)} seconds.")
    elif minutes >= 1:
        print(f"The training goes by {int(minutes)} minutes and {int(seconds)} seconds.")
    else:
        print(f"The training goes by {int(seconds)} seconds.")

## Train and Validation

### Phase I: Train the Classifier Head

Initializing Wandb for Phase I - Train and Validation

In [ ]:
# Initialize WandB
wandb.init(
    project="face-recognition-lfw",
    name="phase1-train-n-validate"
)

Describing hyperparameters used

In [ ]:
# Hyperparameters Phase 1
BATCH_SIZE_phase1 = 64
EPOCHS_phase1 = 150
LEARNING_RATE_phase1 = 3e-4
L2_REGULARIZATION_phase1 = 4e-5

Describing loss function, optimizer, and metrics used when training the model. I use F1-Score because of the imbalance dataset case.

In [ ]:
optimizer_mobilenet_phase1 = AdamW(
    learning_rate=LEARNING_RATE_phase1,
    weight_decay=L2_REGULARIZATION_phase1,
)
loss_function_mobilenet_phase1 = CategoricalCrossentropy()
early_stopping_phase1 = EarlyStopping(monitor='val_f1_score', mode="max", start_from_epoch=10, patience=25, restore_best_weights=True)
scheduler_phase1 = LearningRateScheduler(scheduler)
wandb_phase1 = WandbCallback(save_model=False)

model_mobilenet.compile(
    loss = loss_function_mobilenet_phase1,
    optimizer = optimizer_mobilenet_phase1,
    metrics=[F1Score(average="micro", dtype=np.float32)]
)

Train the model!

In [ ]:
try:
    print("Training MobileNet Phase 1...")
    start_time1 = time.time()
    training_history = model_mobilenet.fit(
        batch_size=BATCH_SIZE_phase1,
        epochs=EPOCHS_phase1,
        x=X1_train,
        y=Y1_train,
        shuffle=True,
        validation_data=(X1_dev, Y1_dev),
        callbacks=[
            early_stopping_phase1,
            scheduler_phase1,
            wandb_phase1,
        ],
        verbose=0 if EPOCHS_phase1 > 50 else 1,
    )
    elapsed_time1 = time.time() - start_time1
    
    plot_training_history(training_history)
    training_time_elapsed(elapsed_time1)
except Exception as e:
    print("Error while training MobileNet!", e)

Evaluate the first phase using development data

In [ ]:
result_eval_mobilenet_phase1 = model_mobilenet.evaluate(X1_dev, Y1_dev, batch_size=BATCH_SIZE_phase1)

In [ ]:
print(f"F1 Score of the MobileNet Phase 1: {round(result_eval_mobilenet_phase1[1] * 100, 2)}%")

The resulting validation vs training F1-Score and its loss is looking good! No sign of overfitting. The initial Phase I training validation F1-Score is 90s%

Finish the current Wandb logging run

In [ ]:
wandb.finish()

### Phase II: Train the 2 Top MobileNet Blocks

Initialize new Wandb run

In [ ]:
# Initialize WandB
wandb.init(
    project="face-recognition-lfw",
    name="phase2-train-n-validate"
)

In [ ]:
print(f"Base model has {len(mobilenet_mdl.layers)} layers.")

Next, I want to train the 2 top MobileNet Convolutional Layers. Unfreeze starting from the 2 top MobileNet Convolutional Layers, and freeze the other layers. This means we unfreeze the last 2 Convolutional Layers (Conv2D), all the way to the last layers. The previous layers should be freezed.

In [ ]:
for layer in model_mobilenet.layers[:len(model_mobilenet.layers)-15]:
    layer.trainable = False
    
for layer in model_mobilenet.layers[len(model_mobilenet.layers)-15:]:
    layer.trainable = True

In [ ]:
# Hyperparameters Phase 2
BATCH_SIZE_phase2 = 32
EPOCHS_phase2 = 100
LEARNING_RATE_phase2 = 1e-4
L2_REGULARIZATION_phase2 = 4e-5

In [ ]:
optimizer_mobilenet_phase2 = AdamW(
    learning_rate=LEARNING_RATE_phase2,
    weight_decay=L2_REGULARIZATION_phase2
)
loss_function_mobilenet_phase2 = CategoricalCrossentropy()
early_stopping_phase2 = EarlyStopping(monitor='val_f1_score', mode="max", start_from_epoch=10, patience=25, restore_best_weights=True)
scheduler_phase2 = LearningRateScheduler(scheduler)
wandb_phase2 = WandbCallback(save_model=False)

model_mobilenet.compile(
    loss = loss_function_mobilenet_phase2,
    optimizer = optimizer_mobilenet_phase2,
    metrics=[F1Score(average="micro", dtype=np.float32)]
)

In [ ]:
try:
    print("Training MobileNet Phase 2...")
    start_time1 = time.time()
    training_history = model_mobilenet.fit(
        batch_size=BATCH_SIZE_phase2,
        epochs=EPOCHS_phase2,
        x=X1_train,
        y=Y1_train,
        shuffle=True,
        validation_data=(X1_dev, Y1_dev),
        callbacks=[
            early_stopping_phase2,
            scheduler_phase2,
            wandb_phase2,
        ],
        verbose=0 if EPOCHS_phase2 > 50 else 1,
    )
    elapsed_time1 = time.time() - start_time1
    
    plot_training_history(training_history)
    training_time_elapsed(elapsed_time1)
except Exception as e:
    print("Error while training MobileNet!", e)

Evaluate the second phase with development data

In [ ]:
result_eval_mobilenet_phase2 = model_mobilenet.evaluate(X1_dev, Y1_dev, batch_size=BATCH_SIZE_phase2)

In [ ]:
print(f"F1 Score of the MobileNet Phase 2: {round(result_eval_mobilenet_phase2[1] * 100, 2)}%")

The resulting training vs validation loss and F1-Score graphs are looking good! No sign of overfitting. The evaluation of the 2nd Phase is the F1-Score of 95s%. This implicates that training the classifier head first then train the 2 last Convolutional Layers improves the resulting F1-Score.

Finish the current Wandb logging run

In [ ]:
wandb.finish()

## Holdout Validation

I use holdout strategy here. I'm going to combine the training and validation/development data to create a new "holdout" data. This holdout data is going to be tested against the testing data.

In [ ]:
# Concatenate the arrays along the first axis
X1_holdout = np.concatenate((X1_train, X1_dev), axis=0)
Y1_holdout = np.concatenate((Y1_train, Y1_dev), axis=0)

### Phase I: Train the Classifier Head

Initialize new Wandb logging run

In [ ]:
# Initialize WandB
wandb.init(
    project="face-recognition-lfw",
    name="phase1-holdout"
)

Re-freeze the model's layers except the last 6 layers.

In [ ]:
for layer in model_mobilenet.layers[:-6]:
    layer.trainable = False

Define hyperparameters for holdout phase 1

In [ ]:
# Hyperparameters holdout Phase 1
BATCH_SIZE_holdout_phase1 = 32
EPOCHS_holdout_phase1 = 20
LEARNING_RATE_holdout_phase1 = 2e-4
L2_REGULARIZATION_holdout_phase1 = 4e-5

Define the loss function, optimizer, callback functions, and compile the model again

In [ ]:
optimizer_mobilenet_holdout_phase1 = AdamW(
    learning_rate=LEARNING_RATE_holdout_phase1,
    weight_decay=L2_REGULARIZATION_holdout_phase1
)
loss_function_mobilenet_holdout_phase1 = CategoricalCrossentropy()
early_stopping_holdout_phase1 = EarlyStopping(monitor='val_f1_score', mode="max", start_from_epoch=10, patience=25, restore_best_weights=True)
scheduler_holdout_phase1 = LearningRateScheduler(scheduler)
wandb_holdout_phase1 = WandbCallback(save_model=False)

model_mobilenet.compile(
    loss = loss_function_mobilenet_holdout_phase1,
    optimizer = optimizer_mobilenet_holdout_phase1,
    metrics=[
        F1Score(average="micro", dtype=np.float32),
    ]
)

Re-train the model using Holdout data

In [ ]:
try:
    print("Training MobileNet Holdout Phase 1...")
    start_time1 = time.time()
    training_history = model_mobilenet.fit(
        batch_size=BATCH_SIZE_holdout_phase1,
        epochs=EPOCHS_holdout_phase1,
        x=X1_holdout,
        y=Y1_holdout,
        shuffle=True,
        validation_data=(X1_test, Y1_test),
        callbacks=[
            early_stopping_holdout_phase1,
            scheduler_holdout_phase1,
            wandb_holdout_phase1,
        ],
        verbose=0,
    )
    elapsed_time1 = time.time() - start_time1
    
    plot_training_history(training_history)
    training_time_elapsed(elapsed_time1)
except Exception as e:
    print("Error while training MobileNet!", e)

In [ ]:
result_eval_mobilenet_holdout_phase1 = model_mobilenet.evaluate(X1_test, Y1_test, batch_size=BATCH_SIZE_holdout_phase1)

In [ ]:
print(f"F1 Score of the MobileNet Holdout Phase 1: {round(result_eval_mobilenet_holdout_phase1[1] * 100, 2)}%")

Finish the current Wandb logging run

In [ ]:
wandb.finish()

### Phase II: Train the 2 Top MobileNet Blocks

Initialize a new Wandb logging run

In [ ]:
# Initialize WandB
wandb.init(
    project="face-recognition-lfw",
    name="phase2-holdout"
)

Unfreeze the base model/MobileNet model except the 2 last Convolutional layers and keep the other layers to be frozen.

In [ ]:
for layer in model_mobilenet.layers[:len(model_mobilenet.layers)-15]:
    layer.trainable = False
    
for layer in model_mobilenet.layers[len(model_mobilenet.layers)-15:]:
    layer.trainable = True

Define the hyperparameters used for Phase 2 of the holdout training

In [ ]:
# Hyperparameters holdout Phase 2
BATCH_SIZE_holdout_phase2 = 32
EPOCHS_holdout_phase2 = 100
LEARNING_RATE_holdout_phase2 = 1e-4
L2_REGULARIZATION_holdout_phase2 = 4e-5

Re-compile the model again

In [ ]:
optimizer_mobilenet_holdout_phase2 = AdamW(
    learning_rate=LEARNING_RATE_holdout_phase2,
    weight_decay=L2_REGULARIZATION_holdout_phase2
)
loss_function_mobilenet_holdout_phase2 = CategoricalCrossentropy()
early_stopping_holdout_phase2 = EarlyStopping(monitor='val_f1_score', mode="max", start_from_epoch=10, patience=20, restore_best_weights=True)
scheduler_holdout_phase2 = LearningRateScheduler(scheduler)
wandb_holdout_phase2 = WandbCallback(save_model=False)

model_mobilenet.compile(
    loss = loss_function_mobilenet_holdout_phase2,
    optimizer = optimizer_mobilenet_holdout_phase2,
    metrics=[
        F1Score(average="micro", dtype=np.float32),
    ]
)

In [ ]:
try:
    print("Training MobileNet Holdout Phase 2...")
    start_time1 = time.time()
    training_history = model_mobilenet.fit(
        batch_size=BATCH_SIZE_holdout_phase2,
        epochs=EPOCHS_holdout_phase2,
        x=X1_holdout,
        y=Y1_holdout,
        shuffle=True,
        validation_data=(X1_test, Y1_test),
        callbacks=[
            early_stopping_holdout_phase2,
            scheduler_holdout_phase2,
            wandb_holdout_phase2
        ],
        verbose=0 if EPOCHS_holdout_phase2 > 50 else 1,
    )
    elapsed_time1 = time.time() - start_time1
    
    plot_training_history(training_history)
    training_time_elapsed(elapsed_time1)
except Exception as e:
    print("Error while training MobileNet!", e)

Evaluating the holdout Phase 2

In [ ]:
result_eval_mobilenet_holdout_phase2 = model_mobilenet.evaluate(X1_test, Y1_test, batch_size=BATCH_SIZE_holdout_phase2)

In [ ]:
print(f"F1 Score of the MobileNet Holdout Phase 2: {round(result_eval_mobilenet_holdout_phase2[1] * 100, 2)}%")

Finalize the current Wandb logging run

In [ ]:
wandb.finish()

# Performance Report & Analysis

## Evaluate

In [ ]:
gc.collect() # garbage collection

Let's test it using the unseen testing data!

In [ ]:
result_mobilenet = model_mobilenet.evaluate(X1_test, Y1_test, batch_size=BATCH_SIZE_holdout_phase2)

## Result

In [ ]:
print(f"F1 Score MobileNet: {round(result_mobilenet[1] * 100, 2)}%")

The result is pretty good!

## Classification Result

In [ ]:
predict_mobilenet = model_mobilenet.predict(X1_test, batch_size=BATCH_SIZE_holdout_phase2)

In [ ]:
Y_pred_1dim = np.argmax(predict_mobilenet, axis=1)

In [ ]:
Y_test_1dim = np.argmax(Y1_test, axis=1)

In [ ]:
print(classification_report(y_true = Y_test_1dim, y_pred = Y_pred_1dim))

In [ ]:
encoder.categories_[0]

In [ ]:
conf_matrice = confusion_matrix(y_true = Y_test_1dim, y_pred = Y_pred_1dim)

In [ ]:
# Define a color map (blue for higher values, red for lower values)
cmap = sns.diverging_palette(240, 10, as_cmap=True)
# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrice, annot=True, fmt=".2f", cmap=cmap, linewidths=.5, square=True)

In [ ]:
final_accuracy = accuracy_score(y_true=Y_test_1dim, y_pred=Y_pred_1dim)
final_f1_score = f1_score(y_true=Y_test_1dim, y_pred=Y_pred_1dim, average="micro")

print(f"Final Accuracy: {round(final_accuracy * 100, 2)}%")
print(f"Final F1 Score: {round(final_f1_score * 100, 2)}%")

# Error Analysis

In [ ]:
true_labels = encoder.categories_[0]
for i_test in range(len(Y_test_1dim)):
    data_in_true = Y_test_1dim[i_test]
    data_in_pred = Y_pred_1dim[i_test]
    
    if data_in_true != data_in_pred:
        print(f"True label: {true_labels[data_in_true]}")
        print(f"Predicted: {true_labels[data_in_pred]}")
        print("===========")

We have the falsely predicted label above and its supposed-to-be-true label. Then, we compare it with the most data from our dataset.

In [ ]:
df_all.sort_values(by="images", ascending=False).head(3)

HA! As expected, our model tends to classify the images as the class that has the largest amount of data: **George W Bush**, **Colin Powell**, and **Tony Blair**.

We could add more data to the other classes to balance them out, or we could implement augmentation techniques to add more data.

# Saving the Model

Initialize Wandb again to log the model

In [ ]:
# Initialize WandB
run = wandb.init(
    project="face-recognition-lfw"
)

In [ ]:
filename = "mobilenet_facerecognition.keras"
model_mobilenet.save(os.path.join(WORK_DIR, filename))

In [ ]:
model_artifact = wandb.Artifact(
  name=filename, 
  type="model"
)
model_artifact.add_file(os.path.join(WORK_DIR, filename))
run.log_artifact(model_artifact)
wandb.finish()